In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import skimage.io
import os
import coco
import yolo
import vgg

In [3]:
repo_dir = project_name = 'COTSDataset' 
part1_single_objects = os.path.join(repo_dir, "Part 1 - Single Objects")
part2_multiple_objects = os.path.join(repo_dir, "Part 2 - Multiple Objects")
part3_complex_background = os.path.join(repo_dir, "Part 3 - Complex Background")

In [4]:
do_display = True
do_save = False
do_print = False

# COCO
coco_output_dir = "COCO_annotations"
coco_color = (0, 255, 0)

# YOLO
yolo_output_dir = "YOLO_annotations"
yolo_color = (0, 255, 0)

# VGG
vgg_output_dir = "VGG_annotations"
vgg_color = 'r'

In [5]:
def iou(annotation):
    gt = annotation['image'][:, :, 0] > 0
    annotation_mask = np.zeros((annotation['height'], annotation['width']), dtype=np.uint8)
    cv2.drawContours(annotation_mask, annotation['contours'], -1, 255, -1)
    # resize annotation mask to gt size
    annotation_mask = cv2.resize(annotation_mask, (gt.shape[1], gt.shape[0]))

    # plot both on each other transparently, one blue one red
    # plt.figure()
    # plt.rcParams['figure.figsize'] = [20, 10]
    # plt.imshow(gt, cmap='Blues', alpha=0.5)
    # plt.imshow(annotation_mask, cmap='Reds', alpha=0.5)
    # plt.xticks([])
    # plt.yticks([])
    # plt.show()

    intersection = np.logical_and(gt, annotation_mask)
    union = np.logical_or(gt, annotation_mask)
    return np.sum(intersection) / np.sum(union)

In [6]:
# Import coco
from pycocotools.coco import COCO
from pycocotools import mask as maskUtils
import skimage.io as io

In [7]:
ann_file = 'Evaluation/instances_train2017.json'
coco_data = COCO(ann_file)

loading annotations into memory...
Done (t=24.47s)
creating index...
index created!


In [8]:
coco_annotations = coco_data.loadAnns(coco_data.getAnnIds())

In [ ]:
# coco_annotations = coco_data.loadAnns(coco_data.getAnnIds())

# # Merge annotations with same image id
# merged_annotations = {}
# for annotation in coco_annotations:
#     # print(merged_annotations)
#     if annotation['image_id'] in merged_annotations:
#         merged_annotations[annotation['image_id']]['segmentation'].extend(annotation['segmentation'])
#     else:
#         merged_annotations[annotation['image_id']] = annotation

In [12]:
from random import randint
rand_id = randint(0, len(coco_annotations))

In [24]:
import skimage.io

In [167]:
id_ = coco_annotations[rand_id]['image_id']
name = coco_data.loadImgs(id_)[0]['file_name']
image = coco_data.annToMask(coco_annotations[rand_id])

original = skimage.io.imread(coco_data.loadImgs(id_)[0]['coco_url'])


im = (id_, name, image, 'COCOEvaluation', 'COCO', './Evaluation/COCO/')

annotation = coco.annotate(im, do_display=False, do_save=True, do_cvt=False, annotation_color=coco_color, epsilon=0.005, configuration=coco.PA)

contours = annotation['contours']

annotated_image = np.zeros((annotation['height'], annotation['width'], 3), dtype=np.uint8)
cv2.drawContours(annotated_image, annotation['contours'], -1, (255, 0, 0), 3, cv2.LINE_AA)


 Annotating image:  000000573320.jpg
Number of contours:  1
 Succesfully saved image:  000000573320.jpg 


------------------------------------------------------------------------------------------------------------------------


AttributeError: module 'cv2' has no attribute 'LINE_AB'

In [18]:
import matplotlib.pyplot as plt

In [166]:
original_mask = image

plt.figure()
plt.rcParams['figure.figsize'] = [20, 10]
plt.subplot(131)
plt.imshow(original)
plt.xticks([])
plt.yticks([])
plt.title('Original Image', fontsize=20)
plt.subplot(132)
image_rgba = cv2.cvtColor(original_mask, cv2.COLOR_GRAY2RGBA)
alpha = np.where(image == 0, 0, 255)
image_rgba[:, :, 0] = 0
image_rgba[:, :, 1] = 0
image_rgba[:, :, 2] = 255 
image_rgba[:, :, 3] = alpha

plt.imshow(original, alpha=1)
plt.imshow(image_rgba, alpha=0.5)

plt.xticks([])
plt.yticks([])
plt.title('Original Mask', fontsize=20)
plt.subplot(133)
# plot contours
plt.imshow(original_mask, cmap='gray')
# plt.imshow(annotated_image)
annotated_rgba = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGBA)
alpha = np.where(annotated_image == 0, 0, 255)
annotated_rgba[:, :, 0] = 255
annotated_rgba[:, :, 1] = 0
annotated_rgba[:, :, 2] = 0
annotated_rgba[:, :, 3] = alpha[:, :, 0]
plt.imshow(annotated_rgba, alpha=1)


plt.xticks([])
plt.yticks([])
plt.title('Annotated Mask', fontsize=20)
plt.show()
plt.savefig('test.jpeg')

c:\Users\matth\anaconda3\envs\coco\lib\site-packages\ipykernel_launcher.py:40: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


In [44]:
cv2.drawContours(annotated_image, annotation['contours'], -1, (255, 0, 0), 3, cv2.LINE_AA)

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [ ]:
annotation['contours']

[]